In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

import cartopy.crs as crs
import cartopy.feature as cfeature
import numpy as np

In [2]:
df_ufo = pd.read_csv("./src/us_ufo_data.csv")
df_airport = pd.read_csv("./src/airports.csv")

# df_airport = df_airport.loc[df_airport["country"] == "USA"]

# df_ufo['description'] = df_ufo['description'].str.capitalize()
# df_ufo['country'] = df_ufo['country'].str.upper() + "A"

# df_ufo.to_csv('./src/us_ufo_data.csv', index=False)
# df_airport.to_csv('./src/airports_clean.csv', index=False)
df_ufo
# df_airport

,date,time,city,state,country,ufo_description,duration,description,latitude,longitude
0,10/10/1949,20:30,San Marcos,TX,USA,cylinder,45 minutes,This event took place in early fall around 194...,29.883056,-97.941111
1,10/10/1956,21:00,Edna,TX,USA,circle,1/2 hour,My older brother and twin sister were leaving ...,28.978333,-96.645833
2,10/10/1960,20:00,Kaneohe,HI,USA,light,15 minutes,As a marine 1st lt. flying an fj4b fighter/att...,21.418056,-157.803611
3,10/10/1961,19:00,Bristol,TN,USA,sphere,5 minutes,My father is now 89 my brother 52 the girl wit...,36.595000,-82.188889
4,10/10/1965,23:45,Norwalk,CT,USA,disk,20 minutes,A bright orange color changing to reddish colo...,41.117500,-73.408333
...,...,...,...,...,...,...,...,...,...,...
64901,9/9/2013,21:15,Nashville,TN,USA,light,10 minutes,Round from the distance/slowly changing colors...,36.165833,-86.784444
64902,9/9/2013,22:00,Boise,ID,USA,circle,20 minutes,Boise id spherical 20 min 10 red lights seen b...,43.613611,-116.202500
64903,9/9/2013,22:00,Napa,CA,USA,other,hour,Napa ufo,38.297222,-122.284444
64904,9/9/2013,22:20,Vienna,VA,USA,circle,5 seconds,Saw a five gold lit cicular craft moving fastl...,38.901111,-77.265556


In [3]:
df_airport

,iata,name,city,state,country,latitude,longitude
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
...,...,...,...,...,...,...,...
3371,ZEF,Elkin Municipal,Elkin,NC,USA,36.280024,-80.786069
3372,ZER,Schuylkill Cty/Joe Zerbey,Pottsville,PA,USA,40.706449,-76.373147
3373,ZPH,Zephyrhills Municipal,Zephyrhills,FL,USA,28.228065,-82.155916
3374,ZUN,Black Rock,Zuni,NM,USA,35.083227,-108.791777


In [4]:
#https://ipython-books.github.io/142-drawing-flight-routes-with-networkx/
names = ('airline,airline_id,'
         'source,source_id,'
         'dest,dest_id,'
         'codeshare,stops,equipment').split(',')
routes = pd.read_csv(
    'https://github.com/ipython-books/'
    'cookbook-2nd-data/blob/master/'
    'routes.dat?raw=true',
    names=names,
    header=None)
routes

,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178,WYA,6334,ADL,3341,NaN,0,SF3
67659,ZM,19016,DME,4029,FRU,2912,NaN,0,734
67660,ZM,19016,FRU,2912,DME,4029,NaN,0,734
67661,ZM,19016,FRU,2912,OSS,2913,NaN,0,734


In [5]:
#https://ipython-books.github.io/142-drawing-flight-routes-with-networkx/
names = ('id,name,city,country,iata,icao,lat,lon,'
         'alt,timezone,dst,tz,type,source').split(',')
airports = pd.read_csv(
    'https://github.com/ipython-books/'
    'cookbook-2nd-data/blob/master/'
    'airports.dat?raw=true',
    header=None,
    names=names,
    index_col=4,
    na_values='\\N')
airports_us = airports[airports['country'] ==
                       'United States']
airports_us

,id,name,city,country,icao,lat,lon,alt,timezone,dst,tz,type,source
iata,,,,,,,,,,,,,
BTI,3411,Barter Island LRRS Airport,Barter Island,United States,PABA,70.134003,-143.582001,2,-9.0,A,America/Anchorage,airport,OurAirports
LUR,3413,Cape Lisburne LRRS Airport,Cape Lisburne,United States,PALU,68.875099,-166.110001,16,-9.0,A,America/Anchorage,airport,OurAirports
PIZ,3414,Point Lay LRRS Airport,Point Lay,United States,PPIZ,69.732903,-163.005005,22,-9.0,A,America/Anchorage,airport,OurAirports
ITO,3415,Hilo International Airport,Hilo,United States,PHTO,19.721399,-155.048004,38,-10.0,N,Pacific/Honolulu,airport,OurAirports
ORL,3416,Orlando Executive Airport,Orlando,United States,KORL,28.545500,-81.332901,113,-5.0,A,America/New_York,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...
XMR,11866,Cape Canaveral AFS Skid Strip,Cocoa Beach,United States,KXMR,28.467600,-80.566597,10,NaN,NaN,NaN,airport,OurAirports
NaN,11867,Homey (Area 51) Airport,Groom Lake,United States,KXTA,37.235001,-115.810997,4462,NaN,NaN,NaN,airport,OurAirports
ZZV,11868,Zanesville Municipal Airport,Zanesville,United States,KZZV,39.944401,-81.892097,900,NaN,NaN,NaN,airport,OurAirports
